# Assignment 3: Build a seq2seq model for machine translation.

### Name: [Matthew Halvorsen]

### Task: Change LSTM model to Bidirectional LSTM Model and Translate English to Spanish

### Due Date: Wednesday, April 17th, 11:59PM

## 0. You will do the following:

1. Read and run the code. Please make sure you have installed keras or tensorflow.Running the script on colab will speed up the training process and also prevent package loading issue.
2. Complete the code in Section 1.1, you may fill in your data directory.
3. Directly modify the code in Section 3. Change the current LSTM layer to a Bidirectional LSTM Model.
4. Training your model and translate English to Spanish in Section 4.2. You could try translating other languages.
5. Complete the code in Section 5.

### Hint:

To implement ```Bi-LSTM```, you will need the following code to build the encoder **in Section 3**. Do NOT use Bi-LSTM for the decoder. But there are other codes **you need to modify** to make it work.

In [ ]:
'''
from keras.layers import Bidirectional, Concatenate

encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, dropout=0.5, name='encoder_lstm'))
 _, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

 state_h = Concatenate()([forward_h, backward_h])
 state_c = Concatenate()([forward_c, backward_c])
 '''

## 1. Data preparation (10 points)

1. Download spanish-english data from http://www.manythings.org/anki/
2. You may try to use other languages.
3. Unzip the .ZIP file.
4. Put the .TXT file (e.g., "deu.txt") in the directory "./Data/".
5. Fill in your data directory in section 1.1.

### 1.1. Load and clean text


In [1]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

#### Fill the following blanks:

In [2]:
# e.g., filename = 'Data/deu.txt'
filename = '/Users/matthewhalvorsen/Desktop/Desktop - Matthew’s MacBook Pro (2)/Grad School /Summer 2025/CS583/Homework/Data/spa.txt'

# e.g., n_train = 20000
n_train = 20000

In [3]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [4]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[tomll know] => [tom lo sabra]
[toms alert] => [tom esta alerta]
[toms alive] => [tom esta vivo]
[toms alone] => [tom esta solo]
[toms angry] => [tom esta furioso]
[toms awake] => [tom esta despierto]
[toms bored] => [tom esta aburrido]
[toms crazy] => [tom es loco]
[toms crazy] => [tom esta loco]
[toms drunk] => [tom esta borracho]


In [5]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (20000,)
Length of target_texts: (20000,)


In [6]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 18
max length of target sentences: 48


**Remark:** To this end, you have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length,
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length,
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (20000, 18)
shape of input_token_index: 27
shape of decoder_input_seq: (20000, 48)
shape of target_token_index: 29


In [8]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


**Remark:** To this end, the input language and target language texts are converted to 2 matrices.

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [9]:
target_texts[100]

'\tentendiste\n'

In [10]:
decoder_input_seq[100, :]

array([ 6,  2,  9,  8,  2,  9, 15, 11,  5,  8,  2,  7,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [11]:
from tensorflow.keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq,
                                    max_decoder_seq_length,
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(20000, 18, 28)
(20000, 48, 30)


## 3. Build the networks (for training) (20 points)

- In this section, we have already implemented the LSTM model for you. You can run the code and see what the code is doing.  

- **You need to change the existing LSTM model to a Bidirectional LSTM model. Just modify the network structure and do not change the training cell in section 3.4.**

- Build encoder, decoder, and connect the two modules to get "model".

- Fit the model on the bilingual data to train the parameters in the encoder and decoder.



### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return:

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Concatenate, Dense

latent_dim = 256

# Inputs to the encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens), name='encoder_inputs')

# Bidirectional LSTM
encoder_bilstm = Bidirectional(
    LSTM(latent_dim, return_state=True, dropout=0.5, name='encoder_lstm'),
    name='bidirectional_encoder_lstm'
)

# Run encoder_inputs through BiLSTM
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

encoder_concat_h = Concatenate(name='encoder_concat_h')([forward_h, backward_h])
encoder_concat_c = Concatenate(name='encoder_concat_c')([forward_c, backward_c])

# Concatenate forward and backward states
state_h = Dense(latent_dim, activation='tanh', name='state_h_reduced')(encoder_concat_h)
state_c = Dense(latent_dim, activation='tanh', name='state_c_reduced')(encoder_concat_c)

# Build encoder model
encoder_model = Model(inputs=encoder_inputs, outputs=[state_h, state_c], name='encoder')

Print a summary and save the encoder network structure to "./encoder.pdf"

In [13]:
from IPython.display import SVG
from keras.utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file='encoder.pdf'
)

encoder_model.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None, 28)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_enco… │ [(None, 512),     │    583,680 │ encoder_inputs[0… │
│ (Bidirectional)     │ (None, 256),      │            │                   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_concat_h    │ (None, 512)       │          0 │ bidirectional_en… │
│ (Concatenate)       │                   │            │ bidirectional_en… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_concat_c    │ (None, 512)       │          0 │ bidirectional_en… │
│ (Concatenate)       │                   │            │ bidirectional_en… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_h_reduced     │ (None, 256)       │    131,328 │ encoder_concat_h… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_c_reduced     │ (None, 256)       │    131,328 │ encoder_concat_c… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 846,336 (3.23 MB)

 Trainable params: 846,336 (3.23 MB)

 Non-trainable params: 0 (0.00 B)

### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$
    
    -- The initial conveyor belt $c_t$

- Return:

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [14]:
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim, return_sequences=True,
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x,
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')

Print a summary and save the encoder network structure to "./decoder.pdf"

In [15]:
from IPython.display import SVG
from keras.utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.pdf'
)

decoder_model.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ decoder_input_x     │ (None, None, 30)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input_h     │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input_c     │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    293,888 │ decoder_input_x[… │
│                     │ 256), (None,      │            │ decoder_input_h[… │
│                     │ 256), (None,      │            │ decoder_input_c[… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 30)  │      7,710 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 301,598 (1.15 MB)

 Trainable params: 301,598 (1.15 MB)

 Non-trainable params: 0 (0.00 B)

### 3.3. Connect the encoder and decoder

In [16]:
# Input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# Get encoder final states (shape: [state_h, state_c] each with (None, 512))
encoder_state_h, encoder_state_c = encoder_model(encoder_input_x)

# Project encoder states from 512 -> 256 to match decoder
from tensorflow.keras.layers import Dense

state_h_reduced = Dense(latent_dim, activation='tanh', name='reduce_state_h')(encoder_state_h)
state_c_reduced = Dense(latent_dim, activation='tanh', name='reduce_state_c')(encoder_state_c)

# Decode
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=[state_h_reduced, state_c_reduced])
decoder_pred = decoder_dense(decoder_lstm_output)

# Build training model
model = Model(inputs=[encoder_input_x, decoder_input_x],
              outputs=decoder_pred,
              name='model_training')

In [17]:
from IPython.display import SVG
from keras.utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.pdf'
)

model.summary()

Model: "model_training"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input_x     │ (None, None, 28)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder             │ [(None, 256),     │    846,336 │ encoder_input_x[… │
│ (Functional)        │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input_x     │ (None, None, 30)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reduce_state_h      │ (None, 256)       │     65,792 │ encoder[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reduce_state_c      │ (None, 256)       │     65,792 │ encoder[0][1]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    293,888 │ decoder_input_x[… │
│                     │ 256), (None,      │            │ reduce_state_h[0… │
│                     │ 256), (None,      │            │ reduce_state_c[0… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 30)  │      7,710 │ decoder_lstm[1][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,279,518 (4.88 MB)

 Trainable params: 1,279,518 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

### 3.4. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [18]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(20000, 18, 28)
shape of decoder_input_data(20000, 48, 30)
shape of decoder_target_data(20000, 48, 30)


In [19]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_split=0.2)

model.save('seq2seq.h5')

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 98ms/step - loss: 1.2773 - val_loss: 1.1216
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 27s 107ms/step - loss: 0.9501 - val_loss: 0.9540
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 27s 108ms/step - loss: 0.8604 - val_loss: 0.8923
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 28s 112ms/step - loss: 0.8307 - val_loss: 0.8669
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 117ms/step - loss: 0.8145 - val_loss: 0.8427
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 30s 121ms/step - loss: 0.7992 - val_loss: 0.8210
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 31s 123ms/step - loss: 0.7861 - val_loss: 0.8139
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 32s 127ms/step - loss: 0.7764 - val_loss: 0.7947
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 31s 124ms/step - loss: 0.7670 - val_loss: 0.7805
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 31s 125ms/step - loss: 0.7556 - val_loss: 0.7701
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 33s 130ms/step - loss: 0.7528 - val_loss: 0.7632
Epoch 12/50
250/250 

## 4. Make predictions

- In this section, you need to complete section 4.2 to translate English to the target language.


### 4.1. Translate English to XXX

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [20]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [21]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])

        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [22]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
-
English:        be punctual
Spanish (true):  se puntual
Spanish (pred):   narar
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
-
English:        be punctual
Spanish (true):  sed puntuales
Spanish (pred):   narar
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/s

### 4.2. Translate an English sentence to the target language （20 points）

1. Tokenization
2. One-hot encode
3. Translate

In [23]:
input_sentence = 'I love you'

max_len=20
input_sequence, word_index = text2sequences(max_len, lines=[input_sentence])
print(input_sequence.shape)

vocab_size = num_encoder_tokens
input_x = onehot_encode(input_sequence, max_len, vocab_size=vocab_size)
print(input_x.shape)

translated_sentence = decode_sequence(input_x)

print('source sentence is: ' + input_sentence)
print('translated sentence is: ' + translated_sentence)

(1, 20)
(1, 20, 28)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
source sentence is: I love you
translated sentence is: ate



# 5. Evaluate the translation using BLEU score

- We have already translated from English to target language, but how can we evaluate the performance of our model quantitatively?

- In this section, you need to re-train the model we built in section 3 and then evaluate the bleu score on testing dataset.

Reference:

https://machinelearningmastery.com/calculate-bleu-score-for-text-python/

https://en.wikipedia.org/wiki/BLEU

#### Hint:

- Randomly partition the dataset to training, validation, and test.

- Evaluate the BLEU score using the test set. Report the average.

- You may use packages to calculate bleu score, e.g., sentence_bleu() from nltk package.

### 5.1. Partition the dataset to training, validation, and test. Build new token index. (10 points)

1. You may try to load more data/lines from text file.
2. Convert text to sequences and build token index using training data.
3. One-hot encode your training and validation text sequences.

In [12]:
from sklearn.model_selection import train_test_split
import numpy as np
n_train2 = 100000
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train2, :]

train_pairs, temp_pairs = train_test_split(clean_pairs, test_size=0.30, random_state=42, shuffle=True)
val_pairs,   test_pairs = train_test_split(temp_pairs,   test_size=0.50, random_state=42, shuffle=True)

input_train = train_pairs[:, 0]
target_train = ['\t' + text + '\n' for text in train_pairs[:, 1]]

input_test = test_pairs[:, 0]
target_test = ['\t' + text + '\n' for text in test_pairs[:, 1]]

input_val = val_pairs[:, 0]
target_val = ['\t' + text + '\n' for text in val_pairs[:, 1]]

max_encoder_seq_length = max(len(line) for line in input_train)
max_decoder_seq_length = max(len(line) for line in target_train)

In [13]:

def text2sequences(max_len, lines, token_index=None):
    if token_index is None:
        tok = Tokenizer(char_level=True, filters='')
        tok.fit_on_texts(lines)
        seqs = tok.texts_to_sequences(lines)
        seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
        return seqs_pad, tok.word_index
    else:
        seqs = [[token_index.get(ch, 0) for ch in s] for s in lines]
        seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
        return seqs_pad

encoder_train_seq, train_token_index = text2sequences(max_encoder_seq_length, input_train, token_index=None)
decoder_train_seq, target_token_index = text2sequences(max_decoder_seq_length, target_train, token_index=None)
encoder_val_int  = text2sequences(max_encoder_seq_length, input_val,  token_index=train_token_index)
encoder_test_int = text2sequences(max_encoder_seq_length, input_test, token_index=train_token_index)

decoder_val_int  = text2sequences(max_decoder_seq_length,  target_val,  token_index=target_token_index)
decoder_test_int = text2sequences(max_decoder_seq_length,  target_test,  token_index=target_token_index)

print('shape of encoder_input_seq: ' + str(encoder_train_seq.shape))
print('shape of input_token_index: ' + str(len(train_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_train_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

num_encoder_tokens = len(train_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

encoder_input_data_tr = onehot_encode(encoder_train_seq, max_encoder_seq_length, num_encoder_tokens)
encoder_input_data_va = onehot_encode(encoder_val_int,   max_encoder_seq_length, num_encoder_tokens)
encoder_input_data_te = onehot_encode(encoder_test_int,  max_encoder_seq_length, num_encoder_tokens)

decoder_input_data_tr = onehot_encode(decoder_train_seq, max_decoder_seq_length, num_decoder_tokens)
decoder_input_data_va = onehot_encode(decoder_val_int,   max_decoder_seq_length, num_decoder_tokens)
decoder_input_data_te = onehot_encode(decoder_test_int,  max_decoder_seq_length, num_decoder_tokens)

# right-shift targets per split (ints -> one-hot)
def shift_left(dec_int):
    tgt = dec_int.copy()
    tgt[:, :-1] = dec_int[:, 1:]
    return tgt

decoder_target_tr = onehot_encode(shift_left(decoder_train_seq), max_decoder_seq_length, num_decoder_tokens)
decoder_target_va = onehot_encode(shift_left(decoder_val_int),   max_decoder_seq_length, num_decoder_tokens)
decoder_target_te = onehot_encode(shift_left(decoder_test_int),  max_decoder_seq_length, num_decoder_tokens)

shape of encoder_input_seq: (70000, 35)
shape of input_token_index: 27
shape of decoder_input_seq: (70000, 68)
shape of target_token_index: 29


### 5.2 Retrain your previous Bidirectional LSTM model with training and validation data and tune the parameters (learning rate, optimizer, etc) based on validation score. (25 points)

1. Use the model structure in section 3 to train a new model with new training and validation datasets.
2. Based on validation BLEU score or loss to tune parameters.

In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Concatenate, Dense
latent_dim = 256

# Inputs to the encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens), name='encoder_inputs')

# Bidirectional LSTM
encoder_bilstm = Bidirectional(
    LSTM(latent_dim, return_state=True, dropout=0.5, name='encoder_lstm'),
    name='bidirectional_encoder_lstm'
)

# Run encoder_inputs through BiLSTM
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

encoder_concat_h = Concatenate(name='encoder_concat_h')([forward_h, backward_h])
encoder_concat_c = Concatenate(name='encoder_concat_c')([forward_c, backward_c])

# Concatenate forward and backward states
state_h = Dense(latent_dim, activation='tanh', name='state_h_reduced')(encoder_concat_h)
state_c = Dense(latent_dim, activation='tanh', name='state_c_reduced')(encoder_concat_c)

# Build encoder model
encoder_model = Model(inputs=encoder_inputs, outputs=[state_h, state_c], name='encoder')

In [16]:
# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim, return_sequences=True,
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x,
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')

In [17]:
# Input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# Get encoder final states (shape: [state_h, state_c] each with (None, 512))
encoder_state_h, encoder_state_c = encoder_model(encoder_input_x)

# Project encoder states from 512 -> 256 to match decoder
from tensorflow.keras.layers import Dense

state_h_reduced = Dense(latent_dim, activation='tanh', name='reduce_state_h')(encoder_state_h)
state_c_reduced = Dense(latent_dim, activation='tanh', name='reduce_state_c')(encoder_state_c)

# Decode
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=[state_h_reduced, state_c_reduced])
decoder_pred = decoder_dense(decoder_lstm_output)

# Build training model
model = Model(inputs=[encoder_input_x, decoder_input_x],
              outputs=decoder_pred,
              name='model_training')
model.summary()


Model: "model_training"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input_x     │ (None, None, 28)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder             │ [(None, 256),     │    846,336 │ encoder_input_x[… │
│ (Functional)        │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input_x     │ (None, None, 30)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reduce_state_h      │ (None, 256)       │     65,792 │ encoder[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reduce_state_c      │ (None, 256)       │     65,792 │ encoder[0][1]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    293,888 │ decoder_input_x[… │
│                     │ 256), (None,      │            │ reduce_state_h[0… │
│                     │ 256), (None,      │            │ reduce_state_c[0… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 30)  │      7,710 │ decoder_lstm[1][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,279,518 (4.88 MB)

 Trainable params: 1,279,518 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
print('shape of encoder_input_data' + str(encoder_input_data_tr.shape))
print('shape of decoder_input_data' + str(decoder_input_data_tr.shape))
print('shape of decoder_target_data' + str(decoder_target_tr.shape))

shape of encoder_input_data(70000, 35, 28)
shape of decoder_input_data(70000, 68, 30)
shape of decoder_target_data(70000, 68, 30)


In [31]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data_tr, decoder_input_data_tr],  # training data
          decoder_target_tr,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_data=([encoder_input_data_va, decoder_input_data_va], decoder_target_va), shuffle=True)

model.save('seq2seq.h5')

Epoch 1/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 231s 211ms/step - loss: 1.1916 - val_loss: 0.8081
Epoch 2/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 247s 226ms/step - loss: 0.9051 - val_loss: 0.7503
Epoch 3/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 251s 230ms/step - loss: 0.8648 - val_loss: 0.7146
Epoch 4/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 254s 233ms/step - loss: 0.8416 - val_loss: 0.6936
Epoch 5/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 259s 236ms/step - loss: 0.8236 - val_loss: 0.6721
Epoch 6/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 255s 233ms/step - loss: 0.8069 - val_loss: 0.6531
Epoch 7/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 255s 233ms/step - loss: 0.7948 - val_loss: 0.6449
Epoch 8/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 257s 235ms/step - loss: 0.7798 - val_loss: 0.6223
Epoch 9/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 260s 238ms/step - loss: 0.7707 - val_loss: 0.6156
Epoch 10/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 265s 242ms/step - loss: 0.7613 - val_loss: 0.6002
Epoch 11/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 260s 237ms/step - loss: 0.7528 - val

### 5.3 Evaluate the BLEU score using the test set. (15 points)

1. Use trained model above to calculate the BLEU score with testing dataset.
2. A reasonable value should be 0.1-0.3. The higher, the better.

In [19]:
# Encoder: from full encoder input -> initial decoder states
encoder_model_infer = Model(
    inputs=encoder_input_x,
    outputs=[state_h_reduced, state_c_reduced]  # the reduced ones you used for training
)
dec_token_in = Input(shape=(1, num_decoder_tokens), name='dec_token_in')  # (B, 1, Vt)
state_h_in   = Input(shape=(latent_dim,), name='state_h_in')
state_c_in   = Input(shape=(latent_dim,), name='state_c_in')

dec_out, h_out, c_out = decoder_lstm(dec_token_in, initial_state=[state_h_in, state_c_in])
step_probs = decoder_dense(dec_out)  # (B, 1, Vt)

decoder_model_infer = Model(
    [dec_token_in, state_h_in, state_c_in],
    [step_probs, h_out, c_out]
)

In [24]:
# run this if the kernal crashed after training is complete
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input
import numpy as np

model = load_model('seq2seq.h5')

# (Optional) Inspect layer names to confirm
print([l.name for l in model.layers])
print("Model inputs:", model.inputs)

# ---- 1) Build encoder inference model (one-hot pipeline)
# Use the actual encoder input tensor from the loaded graph
enc_input_tensor = model.inputs[0]  # assumes [encoder_input_x, decoder_input_x] order

# Grab the reduced states produced in your training graph
state_h_reduced = model.get_layer('reduce_state_h').output
state_c_reduced = model.get_layer('reduce_state_c').output

encoder_model_infer = Model(enc_input_tensor, [state_h_reduced, state_c_reduced], name='encoder_infer')

# ---- 2) Build decoder step model (reuse trained layers)
decoder_lstm  = model.get_layer('decoder_lstm')
decoder_dense = model.get_layer('decoder_dense')
latent_dim    = decoder_lstm.units

# You MUST have num_decoder_tokens in scope (same as training)
# num_decoder_tokens = len(target_token_index) + 1   # ensure this is defined

dec_token_in = Input(shape=(1, num_decoder_tokens), name='dec_token_in')  # one-hot token at t
state_h_in   = Input(shape=(latent_dim,), name='state_h_in')
state_c_in   = Input(shape=(latent_dim,), name='state_c_in')

dec_out, h_out, c_out = decoder_lstm(dec_token_in, initial_state=[state_h_in, state_c_in])
step_probs = decoder_dense(dec_out)

decoder_model_infer = Model([dec_token_in, state_h_in, state_c_in],
                            [step_probs, h_out, c_out],
                            name='decoder_infer')


['encoder_input_x', 'encoder', 'decoder_input_x', 'reduce_state_h', 'reduce_state_c', 'decoder_lstm', 'decoder_dense']
Model inputs: [<KerasTensor shape=(None, None, 28), dtype=float32, sparse=False, ragged=False, name=encoder_input_x>, <KerasTensor shape=(None, None, 30), dtype=float32, sparse=False, ragged=False, name=decoder_input_x>]


In [25]:
# helpers for char-level mapping
id2char_tgt = {i:c for c,i in target_token_index.items()}
start_char  = '\t'
end_char    = '\n'

def one_hot_token(token_id, vocab_size):
    x = np.zeros((1, 1, vocab_size), dtype='float32')
    x[0, 0, token_id] = 1.0
    return x

def greedy_decode_onehot(enc_input_oh, max_len, start_id, end_id):
    # enc_input_oh: (1, max_enc_len, num_encoder_tokens) for ONE sample
    h, c = encoder_model_infer.predict(enc_input_oh, verbose=0)
    token_id = start_id
    decoded = []

    for _ in range(max_len):
        dec_in = one_hot_token(token_id, num_decoder_tokens)
        probs, h, c = decoder_model_infer.predict([dec_in, h, c], verbose=0)
        token_id = int(np.argmax(probs[0, 0, :]))
        if token_id == end_id:
            break
        decoded.append(id2char_tgt.get(token_id, ''))
    return ''.join(decoded)

# figure out IDs of start/end in target vocab
start_id = target_token_index[start_char]
end_id   = target_token_index[end_char]

# decode the whole test set
hyps = []
for i in range(encoder_input_data_te.shape[0]):
    enc_oh = encoder_input_data_te[i:i+1]  # (1, max_enc_len, Vs)
    hyp = greedy_decode_onehot(enc_oh, max_decoder_seq_length, start_id, end_id)
    hyps.append(hyp)

In [26]:
# original cleaned target strings for test split BEFORE adding \t ... \n
# If you only have the '\t... \n' version, strip them back out.
refs = [t.replace('\t','').replace('\n','') for t in target_test]

In [28]:
# pip install sacrebleu
import sacrebleu

# sacrebleu expects list of hypotheses and list of reference lists
bleu = sacrebleu.corpus_bleu(hyps, [refs])
print(f"BLEU = {bleu.score:.2f}")

BLEU = 4.58
